In [ ]:
import iminuit
from importlib import reload
from IPython.display import display, HTML
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

display(HTML("<style>.container { width:95% !important; }</style>"))

import fitting as fit
import mpld3

mpld3.enable_notebook()

rc('figure', figsize=(14, 8))
rc('font', size=12)

# Sky simulation

In [ ]:
#### Simulate input true "sky"
npix = 50
xpix = np.linspace(0, 1, npix)

x0 = 0.5
maxy = 1.
α = -maxy / (x0 - x0**2)
β = -α
amp = 1.
w0 = 0.1
ph = 0.3

truey = α * xpix**2 + β * xpix + amp * np.sin(xpix / w0 + 2 * np.pi * ph)

plot(xpix, truey, label='True Sky')
xlabel('pixel')
ylabel('sky')
legend()
###########

# Without Intercalibrations

## Exact  "Map-making" solution

In [ ]:
# Simulate observations with perfect intercalibrations
ndet = 50
nsamples = 100
sig = 1e-2

H = np.zeros((ndet * nsamples, npix))
for i in range(npix):
    H[:, i] = np.random.randint(0, 2, size=nsamples * ndet)

# Here we assume that all detectors see all pixels a few times and all at the same time
# nnn = nsamples // npix
# H = np.zeros((ndet*nsamples, npix))
# ### We want each pixel to be seen by all detectors
# for i in range(npix):
#     seenpix = np.zeros((ndet, nsamples))
#     for k in range(nnn):
#         seenpix[:, k*npix+i] = 1
#         H[:,i] = np.ravel(seenpix)
#########################

alltod = np.dot(H, truey) + np.random.randn(ndet * nsamples) * sig

# Solve the linear problem
mymap = np.linalg.inv(H.T @ H) @ H.T @ alltod

subplot(1, 3, 1)
plot(xpix, truey, label='True Sky')
plot(xpix, mymap, 'ro', label='reconstructed')
xlabel('pixel')
ylabel('sky')
legend()

subplot(1, 3, 2)
plot(xpix,
     mymap - truey, color='b',
     label='Residuals σ={0:5.3g}'.format(np.std(mymap - truey)))
xlabel('pixel')
ylabel('sky')
legend()

subplot(1, 3, 3)
a = fit.myhist(mymap - truey,  color='b',label='Residuals')


tight_layout()

## $\chi^2$ in TOD domain

Same problem resolution but with aconjugate gradient (Minuit)
=> Philosophically what we do with QUBIC...

In [ ]:
class simulate_signal:

    def __init__(self, xpix, ndet, nsamples, H):
        self.xpix = xpix
        self.ndet = ndet
        self.nsamples = nsamples
        self.H = H

    def __call__(self, x, sky):
        alltod = np.dot(self.H, sky)
        return alltod


myH = simulate_signal(xpix, ndet, nsamples, H)

guess_sky = np.zeros(len(xpix))
xxx = np.zeros(len(alltod))  # not used
data = fit.Data(xxx, alltod, xxx * 0 + 1, myH)

m, ch2, ndf = data.fit_minuit(guess_sky + 1)

mymap2 = m.values

subplot(1, 3, 1)
plot(xpix, truey, label='True Sky')
plot(xpix, mymap, 'ro', label='reconstructed')
xlabel('pixel')
ylabel('sky')
legend()

subplot(1, 3, 2)
plot(xpix,
     mymap - truey, color='b',
     label='Residuals Analytical σ={0:5.3g}'.format(np.std(mymap - truey)))
plot(xpix,
     mymap2 - truey, color='r',
     label='Residuals Minuit σ={0:5.3g}'.format(np.std(mymap - truey)))
xlabel('pixel')
ylabel('sky')
legend()

subplot(1, 3, 3)
a = fit.myhist(mymap - truey,  color='b',label='Residuals Analytical')
a = fit.myhist(mymap2 - truey,  color='r',label='Residuals Minuit')


tight_layout()

## $\chi2$ in map domain
Here we try to do as in `QubicSoft` reconstruction: we solve the equation $$A\cdot\vec{x}=\vec{b}$$ where:
$$A= (H^t N^{-1} H)$$
and
$$b = H^t N^{-1} \vec{d}$$

In [ ]:
reload(fit)
import iminuit


class MyChi2MapDomain:
    """
    Ridge Chi2
    """

    errordef = iminuit.Minuit.LEAST_SQUARES  # for Minuit to compute errors correctly

    def __init__(self, A,b):
        self.b = b
        self.A = A

    def __call__(self, pars):
        ch2 = np.sum( (self.A @ pars - self.b)**2)
        return ch2

    @property
    def ndata(self):
        return len(self.x)
    
A = H.T @ H
b = H.T @ alltod
mychi2 = MyChi2MapDomain(A, b)
guess_sky = np.zeros(len(xpix))

m = iminuit.Minuit(mychi2, guess_sky)
m.migrad()  
m.hesse() 

mymap3 = m.values


subplot(1, 3, 1)
plot(xpix, truey, label='True Sky')
plot(xpix, mymap2, 'ro', label='reconstructed')
xlabel('pixel')
ylabel('sky')
legend()

subplot(1, 3, 2)
plot(xpix,
     mymap - truey, color='b',
     label='Residuals Analytical (TOD domain) σ={0:5.3g}'.format(np.std(mymap - truey)))
plot(xpix,
     mymap2 - truey, color='r',
     label='Residuals Minuit (TOD domain) σ={0:5.3g}'.format(np.std(mymap2 - truey)))
plot(xpix,
     mymap3 - truey, color='g',
     label='Residuals Mnuit (Map domain) σ={0:5.3g}'.format(np.std(mymap2 - truey)))
xlabel('pixel')
ylabel('sky')
legend()

subplot(1, 3, 3)
a = fit.myhist(mymap - truey,  color='b',label='Residuals Analytical (TOD Domain)')
a = fit.myhist(mymap2 - truey,  color='r',label='Residuals Minuit (TOD Domain)')
a = fit.myhist(mymap3 - truey,  color='g',label='Residuals Minuit (Map Domain)')


tight_layout()


# With inter-calibrations
We will fit them all together with the sky - We also put a non uniform noise on the different detectors

In [ ]:
np.random.seed(42)
###### Simulate observations including intercalibration
ndet = 50
nsamples = 100

###### Noise:
# Uniform noise
# sig = np.zeros(ndet) + 1e-2
# Variable noise across detectors
sig = 1e-2 * np.random.rand(ndet)

H = np.zeros((ndet * nsamples, npix))
for i in range(npix):
    H[:, i] = np.random.randint(0, 2, size=nsamples * ndet)

intercal = np.append([1], (np.random.rand(ndet - 1) + 0.5))
alltod_intercal = np.dot(H, truey)
alltod = alltod_intercal.copy()
alltod_noiseless = alltod_intercal.copy()
for k in range(ndet):
    alltod_noiseless[k * nsamples:(k + 1) * nsamples] *= intercal[k]
    alltod[k * nsamples:(k + 1) * nsamples] *= intercal[k]
    alltod[k * nsamples:(k + 1) *
           nsamples] += np.random.randn(nsamples) * sig[k]
#########################

sigmas = np.ones(len(alltod))
for k in range(ndet):
    sigmas[k * nsamples:(k + 1) * nsamples] *= sig[k]

chi2_noiseless = np.sum(((alltod - alltod_noiseless) / sigmas)**2)
print('Chi2 Noiseless: ', chi2_noiseless)

## $\chi^2$ in TOD domain

### Here we fully sove with Minuit

In [ ]:
class simulate_signal_intercal:

    def __init__(self, xpix, ndet, nsamples, H):
        self.xpix = xpix
        self.ndet = ndet
        self.nsamples = nsamples
        self.H = H

    def __call__(self, x, sky_and_intercal):
        sky = sky_and_intercal[:len(self.xpix)]
        intercal = sky_and_intercal[len(self.xpix):]
        alltod = np.dot(self.H, sky)
        for k in range(self.ndet):
            alltod[k * self.nsamples:(k + 1) * self.nsamples] *= intercal[k]
        return alltod


myH = simulate_signal_intercal(xpix, ndet, nsamples, H)

guess_sky = np.append(np.zeros(len(xpix)), np.ones(ndet))

xxx = np.zeros(len(alltod))  # not used
data = fit.Data(xxx, alltod, sigmas, myH)
m, ch2, ndf = data.fit_minuit(guess_sky, fixpars=[len(xpix)])
mytod_th = myH(0, m.values)
chi2_ana = np.sum(((mytod_th - alltod) / sigmas)**2)

print('Minuit says : chi2={} ndf={}'.format(ch2, ndf))
print('============================')
print('My Chi2 = {}'.format(chi2_ana))
print('The number of unknowns is: {}'.format(npix + ndet - 1))
print('The number of points is: {}'.format(len(alltod)))
print('The ndf={}'.format(len(alltod) - (npix + ndet - 1)))
print('============================')
print('With noiseless true TOD')
chi2_true = np.sum(((alltod_noiseless - alltod) / sigmas)**2)
print('Chi2 True = {}'.format(chi2_true))

mymap = m.values[:len(xpix)]
myintercals = m.values[len(xpix):]

subplot(2, 3, 1)
plot(xpix, truey, label='True Sky')
plot(xpix, mymap, 'ro', label='reconstructed')
xlabel('pixel')
ylabel('sky')
legend()

subplot(2, 3, 2)
plot(xpix,
     mymap - truey, color='b',
     label='Full Minuit Residuals  (TOD DOmain) σ={0:5.3g}'.format(np.std(mymap - truey)),
     alpha=0.5)
xlabel('pixel')
ylabel('sky')
legend(fontsize=10)

subplot(2, 3, 3)
a = fit.myhist(mymap - truey,  color='b',label='Full Minuit Residuals (TOD DOmain) ')

subplot(2, 3, 4)
plot(intercal, label='True Intercals')
plot(myintercals, 'ro', label='reconstructed intercals')
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2, 3, 5)
plot(myintercals - intercal, color='b',
     label='Full Minuit Residuals  (TOD DOmain) σ={0:5.3g}'.format(
         np.std(myintercals - intercal)),
     alpha=0.5)
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2, 3, 6)
a = fit.myhist(myintercals - intercal,  color='b',label='Full Minuit Residuals (TOD Domain)')

tight_layout()

print()
print('RMS Residuals')
print('         M')
print('   Map.        Inter     ')
print('  {0:5.3g}   {1:5.3g}'.format(np.std(mymap - truey),
                                     np.std(myintercals - intercal)))

### Fit inter-calibration analytically
We do not fit the inter-calibrations with the conjugate gradient but we use the fact that they are linear with the data to fit them analytically at each step of the conjugate gradient.

<span class="mark">We should get the same performance, but for now, we don't...</span>

We have data from $n_d$ detectors that observe a sky $\vec{s}$ through an instrument modeled by the operator $H$. Each detector has its own intercalibration.
So the perfectly intercalibrated data writes:
\begin{equation}
\vec{D} = H\vec{s} + \vec{n}
\end{equation}

And the maximum likelihood solution would be:
\begin{equation}
\hat{\vec{s}} = \left(H^t N^{-1} H\right)^{-1} H^t N^{-1} \vec{D}
\end{equation}
    
But in reality we measure $\vec{d}$ which is not intercalibrated:
\begin{equation}
\vec{d}=\vec{D} A
\end{equation}
where A is unknown...

So we proceed with the "inverse problem" approach:
\begin{equation}
\vec{d}=\vec{D} A = H \vec{s} A+\vec{n}
\end{equation}

We vary $A$ and $\vec{s}$ and minimize:
\begin{equation}
\chi^2(A,\vec{s}) = \left(H\vec{s}A-\vec{d}\right)^t N^{-1}\left(H\vec{s}A-\vec{d}\right)
\end{equation}
the solution is obtained by solving:
\begin{equation}
\frac{d\chi^2}{dA^t} = 0 = \vec{s}^t H^t N^{-1} H \vec{s}A - \vec{s}^t H^t N^{-1}\vec{d}
\end{equation}
\begin{equation}
\Rightarrow~~~~ A= \left(\vec{s}^t H^t N^{-1} H \vec{s}\right)^{-1} \vec{s}^t H^t N^{-1}\vec{d}
\end{equation}

So, at each step of the conjugate gradient (at each call to Minuit), we will test a sky $\vec{s}_i$ that correpsond to "test"-intercalibrated data $\vec{D}_i = H\vec{s}_i$. We will compute at each of those steps:
\begin{equation}
A_i = \left(\vec{D}_i^t N^{-1}\vec{D}_i\right) \vec{D}_i^t N^{-1} \vec{d}
\end{equation}
which allows to compute the corresponding $\chi^2$ as:
\begin{equation}
 \chi^2 = \left(\vec{D}_i A_i-\vec{d}\right)^t N^{-1}\left(\vec{D}_i A_i -\vec{d}\right)
\end{equation}
and we minimize it only with respect to $\vec{s}$.

Once we have converged on $\hat{\vec{s}}$, the intercalibrations will be obtained by:
\begin{equation}
 \hat{\vec{D}} = H\hat{\vec{s}}
\end{equation}
and
\begin{equation}
 \hat{A} = \left(\hat{\vec{D}}^t N^{-1}\hat{\vec{D}}\right)^{-1} \hat{\vec{D}}^t N^{-1} \vec{d}
\end{equation}

#### In practice:
If $N$ is dense it is possibly a large problem...

- if $N\propto \mathbb{1}$ the problem is completely simplified and $N$ disappears:
    - $\vec{D}_i = H\vec{s}_i$
    - $A_i = \left(\vec{D}^t\vec{D}\right)^{-1}\vec{D}^t \vec{d}$
    - $\chi^2 = \| \vec{D}_i A_i -\vec{d}\|^2$
    
- If $N$ is diagonal: $N^{-1} = \vec{w}\mathbb{1}$
    - $\vec{D}_i = H\vec{s}_i$
    - $A_i = \left(\vec{D}^t\vec{w}\vec{D}\right)^{-1}\vec{D}^t \vec{w}\vec{d}$
    - $\chi^2 = \|\vec{\sqrt{w}}\cdot\left(\vec{D}_i A_i -\vec{d}\right)\|^2$


In [ ]:
reload(fit)


class simulate_signal_intercal:

    def __init__(self, xpix, ndet, nsamples, H, d, w):
        self.xpix = xpix
        self.ndet = ndet
        self.nsamples = nsamples
        self.H = H
        self.d = np.reshape(d, (ndet, nsamples))
        self.w = w
        self.intercal = None

    def __call__(self, x, sky):
        # get the intercalibrated TOD directly from the sky + the calculated intercalibrations
        D, intercal = self.give_intercals(sky)
        # apply the intercalibrations
        d = (D.T * intercal).T
        return np.ravel(d)

    def give_intercals(self, sky):
        ### We remove the first detector from the fit as its intercalibration is forced to 1
        D = (np.reshape(np.dot(self.H, sky), (self.ndet, self.nsamples)))
        myD = D[1:, :]
        myw = np.diag(self.w[1:]) 
        myDw = (myD.T @ myw).T
        mydw = (self.d[1:, :].T @ myw).T
        print('myDw', np.shape(myDw))
        print('mydw', np.shape(mydw))
        print('\n np.sum(myD*myDw, axis=1)', np.shape(np.sum(myD*myDw, axis=1)), np.sum(myD*myDw, axis=1))
        
        truc = np.linalg.inv(myDw.T @ myD) @ myD.T @ mydw
        print('\n res', np.shape(truc), truc)
        
        bla = 1/np.sum(myD*myDw, axis=1) * np.sum(myD * mydw, axis=1)
        print('\n oldres', np.shape(bla), bla)
        stop
        self.intercal = np.append(
             1,  1/np.sum(myD*myDw, axis=1) * np.sum(myD * mydw, axis=1))
        return D, self.intercal


sigmas = np.ones(len(alltod))
for k in range(ndet):
    sigmas[k * nsamples:(k + 1) * nsamples] *= sig[k]

#### Fit
myH = simulate_signal_intercal(xpix, ndet, nsamples, H, alltod, 1. / sig**2)
guess_sky = np.ones(len(xpix))
xxx = np.zeros(len(alltod))  # not used
data = fit.Data(xxx, alltod, sigmas, myH)
m, ch2, ndf = data.fit_minuit(guess_sky)
mytod_th = myH(0, m.values)
chi2_ana = np.sum(((mytod_th - alltod) / sigmas)**2)

print('Minuit says : chi2={} ndf={}'.format(ch2, ndf))
print(
    'But we know it forgets the {} variables we fit analytically, so it should say {}'
    .format(ndet - 1, ndf - (ndet - 1)))
print('============================')
print('My Chi2 = {}'.format(chi2_ana))
print('The number of unknowns is: {}'.format(npix + ndet - 1))
print('The number of points is: {}'.format(len(alltod)))
print('The ndf={}'.format(len(alltod) - (npix + ndet - 1)))
print('============================')
print('With noiseless true TOD')
chi2_true = np.sum(((alltod_noiseless - alltod) / sigmas)**2)
print('Chi2 True = {}'.format(chi2_true))

mymap2 = np.array(m.values)
myintercals = myH.intercal
myresult2, myintercals2 = myH.give_intercals(m.values)

subplot(2, 3, 1)
plot(xpix, truey, label='True Sky')
plot(xpix, mymap2, 'ro', label='reconstructed')
xlabel('pixel')
ylabel('sky')
legend()

subplot(2, 3, 2)
plot(xpix,
     mymap - truey, color='b',
     label='Full Minuit Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap - truey)),
     alpha=0.5)
plot(xpix,
     mymap2 - truey, color='r',
     label='Minuit + Ana Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap2 - truey)))
xlabel('pixel')
ylabel('sky')
legend(fontsize=10)

subplot(2, 3, 3)
a = fit.myhist(mymap - truey,  color='b',label='Full Minuit Residuals (TOD Domain)')
a = fit.myhist(mymap2 - truey,  color='r',label='Minuit + Ana Residuals (TOD Domain)')

subplot(2, 3, 4)
plot(intercal, label='True Intercals')
plot(myintercals2, 'ro', label='reconstructed intercals')
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2, 3, 5)
plot(myintercals - intercal, color='b',
     label='Full Minuit Residuals  (TOD Domain)σ={0:5.3g}'.format(
         np.std(myintercals - intercal)),
     alpha=0.5)
plot(myintercals2 - intercal, color='r',
     label='Minuit + Ana Residuals  (TOD Domain)σ={0:5.3g}'.format(
         np.std(myintercals2 - intercal)))
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2, 3, 6)
a = fit.myhist(myintercals - intercal,  color='b',label='Full Minuit Residuals (TOD Domain)')
a = fit.myhist(myintercals2 - intercal,  color='r',label='Minuit + Ana Residuals (TOD Domain)')

tight_layout()

print()
print('RMS Residuals')
print('                   M+A                    M')
print('              Map.      Inter.      Map.        Inter     ')
print('            {0:5.3g}   {1:5.3g}    {2:5.3g}   {3:5.3g}'.format(
    np.std(mymap2 - truey), np.std(myintercals2 - intercal),
    np.std(mymap - truey), np.std(myintercals - intercal)))

In [ ]:
debug

### Ridge Likelihood
We use the method proposed in [Stompor et al., 2009] (https://arxiv.org/pdf/0804.2645.pdf) for maximum-likelihood estimation of component separation. Here, intercalibrations are the analogous of the spectral behavior of the astrophysical components. 

The method is the following:
We have data from $n_d$ detectors that observe a sky $\vec{s}$ through an instrument modeled by the operator $H$. Each detector has its own intercalibration $A$.

The perfectly intercalibrated data writes:
\begin{equation}
\vec{D} = H\vec{s} + \vec{n}
\end{equation}
    
But in reality we measure $\vec{d}$ which is not intercalibrated:
$\vec{d}=A\vec{D} $ where A is unknown...

So we have:
    \begin{eqnarray}
        \vec{d} &=& A H\vec{s} + \vec{n} \\
                &=& B \vec{s} + \vec{N}
    \end{eqnarray}

The LogLikelihood writes:
    \begin{eqnarray}
        \chi^2(A, \vec{s}) = -2 \ln \mathcal{L}(A, \vec{s}) &=& \left(\vec{d}-AH\vec{s}\right)^t N^{-1}\left(\vec{d}-AH\vec{s}\right)\\
        &=& \left(\vec{d}-B\vec{s}\right)^t N^{-1}\left(\vec{d}-B\vec{s}\right)
    \end{eqnarray}

With formal solution for the sky $\vec{s}$:
    \begin{equation}
        \hat{\vec{s}} = \left(B^t N^{-1} B\right)^{-1} B^t N^{-1} \vec{d}
    \end{equation}

Now the trick is to re-inject this solution into the LogLikelihood in order to have a Likelihood that only depends upon $A$ (through $B$):
    \begin{eqnarray}
        \chi^2(A) = -2 \ln \mathcal{L}(A) &=& \left(\vec{d}-B\hat{\vec{s}}\right)^t N^{-1}\left(\vec{d}-B\hat{\vec{s}}\right) \\
        &=& ... \\
        &=& \vec{d}^t N^{-1}\vec{d} - \left(B^t N^{-1} \vec{d}\right)^t \left(B^t N^{-1} B\right)^{-1} \left(B^t N^{-1} \vec{d}\right) \\
        &=& \vec{d}^t N^{-1}\vec{d} - \left(H^t A^t N^{-1} \vec{d}\right)^t \left(H^t A^t N^{-1} AH\right)^{-1} \left(H^t A^t N^{-1} \vec{d}\right) 
    \end{eqnarray}
which, as expected only depends upon the intercalibrations $A$.

We therefore minimize this $\chi^2$ using `Minuit` and once we have the bets-fit $\hat{A}$, we can calculate the sky through:
    \begin{equation}
        \hat{\vec{s}} = \left(H^t \hat{A}^t N^{-1} \hat{A}H\right)^{-1} \left(H^t \hat{A}^t N^{-1} \vec{d}\right)
    \end{equation}

In [ ]:
reload(fit)
import iminuit


class RidgeChi2:
    """
    Ridge Chi2
    """

    errordef = iminuit.Minuit.LEAST_SQUARES  # for Minuit to compute errors correctly

    def __init__(self, x, y, err, model):
        self.model = model  # model predicts y for given x
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        self.err = np.asarray(err)
        self.w = 1. / self.err**2
        self.dNid = self.y.T * self.w @ self.y  # d^t N^{-1} d

    def __call__(self, *par):
        B = self.model(self.x, *par)  # B = A.H
        BtNiB = B.T * self.w @ B  # B^t N^{-1} B as noise is uncorrelated across detectors
        invBtNiB = np.linalg.inv(BtNiB)  # (B^t N^{-1} B)^{-1}
        BtNid = B.T * self.w @ self.y  # (B^t N^{-1} d)
        ch2 = self.dNid - BtNid.T @ invBtNiB @ BtNid  # the chi2
        return ch2

    @property
    def ndata(self):
        return len(self.x)


class giveB:

    def __init__(self, H, ndet, nsamples):
        self.ndet = ndet
        self.nsamples = nsamples
        self.H = H

    def __call__(self, x, intercals):
        return (np.repeat(intercals, self.nsamples).T * self.H.T).T


sigmas = np.ones(len(alltod))
for k in range(ndet):
    sigmas[k * nsamples:(k + 1) * nsamples] *= sig[k]

#### Fit
myB = giveB(H, ndet, nsamples)
guess_intercal = np.ones(ndet)

xxx = np.zeros(len(alltod))  # not used
rc2 = RidgeChi2(xxx, alltod, sigmas, myB)

data = fit.Data(xxx, alltod, sigmas, myB)
guess_sky = np.ones(npix)
m, ch2, ndf = data.fit_minuit(guess_sky, minimizer=RidgeChi2, fixpars=[0])

ridge_intercals = np.array(m.values)

### Now get the sky
bestB = myB(0, ridge_intercals)
bestSky = (np.linalg.inv(bestB.T * 1. / sigmas**2 @ bestB) @ bestB.T * 1. /
           sigmas**2 @ alltod)

print('Minuit valid fit:', m.valid)

subplot(2,3,1)
plot(xpix, truey, label='True Sky')
plot(xpix, bestSky, 'ro', label='reconstructed')
xlabel('pixel')
ylabel('sky')
legend()

subplot(2,3,2)
plot(xpix, mymap - truey, color='b', label='Full Minuit Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap - truey)), alpha=0.5)
plot(xpix, mymap2 - truey, color='r', label='Minuit + Ana Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap2 - truey)))
plot(xpix, bestSky - truey, color='g', label='Ridge Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap - truey)), alpha=0.5)
xlabel('pixel')
ylabel('sky')
legend(fontsize=10)

subplot(2,3,3)
a = fit.myhist(mymap - truey, color='b', label='Full Minuit Residuals (TOD Domain)')
a = fit.myhist(mymap2 - truey, color='r', label='Minuit + Ana Residuals (TOD Domain)')
a = fit.myhist(bestSky - truey, color='g', label='Ridge Residuals (TOD Domain)')

subplot(2,3,4)
plot(intercal, label='True Intercals')
plot(ridge_intercals, 'ro', label='reconstructed intercals')
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2,3,5)
plot(myintercals-intercal, color='b', label='Full Minuit Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(myintercals-intercal)), alpha=0.5)
plot(myintercals2-intercal, color='r', label='Minuit + Ana Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(myintercals2-intercal)))
plot(ridge_intercals-intercal, color='g', label='Ridge Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(ridge_intercals-intercal)), alpha=0.5)
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2,3,6)
a = fit.myhist(myintercals-intercal, color='b', label='Full Minuit Residuals (TOD Domain)')
a = fit.myhist(myintercals2-intercal, color='r', label='Minuit + Ana Residuals (TOD Domain)')
a = fit.myhist(ridge_intercals-intercal, color='g', label='Ridge Residuals (TOD Domain)')

## $\chi^2$ in Map domain (Full Minuit)
Does not work for some reason... but this is an absurd way of doing things...

In [ ]:
reload(fit)
import iminuit


class MyChi2MapDomain:
    """
    Chi2 for Map-Domain fitting of map+intercals
    """
    errordef = iminuit.Minuit.LEAST_SQUARES  # for Minuit to compute errors correctly

    def __init__(self, y, err, H, npix, ndet, nsamples):
        self.npix = npix
        self.ndet = ndet
        self.nsamples = nsamples
        self.H = H  # model predicts y for given x
        self.y = np.asarray(y)
        self.err = np.asarray(err)
        self.w = 1. / self.err**2

    def __call__(self, pars):
        thesky = pars[:npix]
        theintercal = pars[npix:]
        B = self.giveB(theintercal)   # B = A.H
        BtNiB = B.T * self.w @ B  # B^t N^{-1} B as noise is uncorrelated across detectors
        BtNid = B.T * self.w @ self.y  # (B^t N^{-1} d)
        ch2 = np.sum( (BtNiB @ thesky - BtNid)**2 )
        return ch2
    
    def giveB(self, intercals):
        return (np.repeat(intercals, self.nsamples).T * self.H.T).T

    @property
    def ndata(self):
        return len(self.x)

    
# ### Test: avec TOD_noiseless ca donne bien un chi2~0 ###
# theB = (np.repeat(intercal, nsamples).T * H.T).T
# BtNiB = theB.T * (1./sigmas**2) @ theB  # B^t N^{-1} B as noise is uncorrelated across detectors
# BtNid = theB.T * (1./sigmas**2) @ alltod_noiseless  # (B^t N^{-1} d)
# mych2 = np.sum( (BtNiB @ truey - BtNid)**2 )
# print(mych2)


xxx = np.zeros(len(alltod))  # not used
mychi2 = MyChi2MapDomain(alltod, sigmas, H, npix, ndet, nsamples)
guess = np.ones(npix+ndet)
# guess = np.append(truey/2, np.ones(ndet)/2)
# guess = np.append(truey*0.9, intercal*0.9)
guess[npix] =1


print(mychi2(guess))


m = iminuit.Minuit(mychi2, guess)
### Fix the first intercal to 1
fixpars = [npix]
for k in range(len(fixpars)):
    m.fixed["x{}".format(fixpars[k])]=True
m.migrad()  
m.hesse() 

mymap4 = m.values[:len(xpix)]
myintercals4 = m.values[len(xpix):]


subplot(2, 3, 1)
plot(xpix, truey, label='True Sky')
plot(xpix, mymap4, 'ro', label='reconstructed')
xlabel('pixel')
ylabel('sky')
legend()

subplot(2,3,2)
plot(xpix, mymap - truey, color='b', label='Full Minuit Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap - truey)), alpha=0.5)
plot(xpix, mymap2 - truey, color='r', label='Minuit + Ana Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap2 - truey)))
plot(xpix, bestSky - truey, color='g', label='Ridhe Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(mymap - truey)), alpha=0.5)
plot(xpix, mymap3 - truey, color='m', label='Full Minuit Residuals (map Domain) σ={0:5.3g}'.format(np.std(mymap - truey)), alpha=0.5)
xlabel('pixel')
ylabel('sky')
legend(fontsize=10)

subplot(2,3,3)
a = fit.myhist(mymap - truey, color='b', label='Full Minuit Residuals (TOD Domain)')
a = fit.myhist(mymap2 - truey, color='r', label='Minuit + Ana Residuals (TOD Domain)')
a = fit.myhist(bestSky - truey, color='g', label='Ridge Residuals (TOD Domain)')
a = fit.myhist(mymap4 - truey, color='m', label='Full Minuit Residuals (Map Domain)')

subplot(2,3,4)
plot(intercal, label='True Intercals')
plot(myintercals4, 'ro', label='reconstructed intercals')
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2,3,5)
plot(myintercals-intercal, color='b', label='Full Minuit Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(myintercals-intercal)), alpha=0.5)
plot(myintercals2-intercal, color='r', label='Minuit + Ana Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(myintercals2-intercal)))
plot(ridge_intercals-intercal, color='g', label='Ridge Residuals (TOD Domain) σ={0:5.3g}'.format(np.std(ridge_intercals-intercal)), alpha=0.5)
plot(myintercals4-intercal, color='m', label='Full Minuit Residuals (Map Domain) σ={0:5.3g}'.format(np.std(myintercals-intercal)), alpha=0.5)
xlabel('Det')
ylabel('Intercal')
legend()

subplot(2,3,6)
a = fit.myhist(myintercals-intercal, color='b', label='Full Minuit Residuals (TOD Domain)')
a = fit.myhist(myintercals2-intercal, color='r', label='Minuit + Ana Residuals (TOD Domain)')
a = fit.myhist(ridge_intercals-intercal, color='g', label='Ridge Residuals (TOD Domain)')
a = fit.myhist(myintercals4-intercal, color='m', label='Full Minuit Residuals (Map Domain)')
tight_layout()

print(m.valid)

In [ ]:
myintercals4

In [ ]:
g = np.array([[1,2]]).T
print(np.shape(g))
print(g)
print()
M = np.array([[1,1,1,0,0,0],[0,0,0,1,1,1]]).T
print(np.shape(M))
print(M)
print()

G = M@g
print(np.shape(G))
print(G)
print()

D = np.array([[0.5, 1, 1.5, 0.3, 0.4, 0.5]]).T
print(D)
print()

d = G * D
print(d)
print()

In [ ]:
G2 = np.diag(M@g[:,0])
G2 @ D

In [ ]:
D = np.array([[0.5, 1, 1.5],[0.3, 0.4, 0.5]]).T
print(D)
print()

M = np.array([[1,1,1],[1,1,1]]).T
print(M)
print()

print(g)
print()

print(M@M.T)